In [3]:
import numpy as np
import os
import netCDF4 as nc
from scipy.spatial import distance_matrix
from sklearn.cluster import MeanShift
from collections import Counter
from scipy.spatial.distance import euclidean
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors.kde import KernelDensity
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
import debacl as dcl
from datetime import datetime, timedelta
X=np.array([[1.5,2,3],[2.5,5,8],[5.5,10,11]])
Y=np.array([[2,3,4],[2.5,3.5,4.5],[5.5,6,7],[6,7,8]])
a=distance_matrix(X,X)

In [4]:
#get the file path for loading, data file is under the same dir with the notebook
filename="20121015_00_ecmwf_ensemble_forecast.PRESSURE_LEVELS.EUR_LL10.120.pl.nc"
foldername="ECWMF Datasets"
filepath=os.path.join(os.path.dirname(os.getcwd()),foldername,filename)

# read the raw data and extract the needed data
# exrtact the value of Geopotential under the pressure of 500 hPA in the certain
Pressure_Levels_data = nc.Dataset(filepath,"r")
g = 9.80655
# get all the dimension value
nd_1,nd_2,nd_3,nd_4,nd_5 = Pressure_Levels_data.variables['Geopotential_isobaric'][:].shape
# get the necessary raw data
Geopotential_Isobaric_500 = Pressure_Levels_data.variables['Geopotential_isobaric'][0,:,7,:,:]/g
# reshape the dataset into form of (51,41*101)
Geopotential_Isobaric_500_reshaped = np.reshape(Geopotential_Isobaric_500,(nd_2, nd_4 * nd_5))
# prepare the longitude and latitude value for contour
longitude = Pressure_Levels_data['lon'][:]
latitude = Pressure_Levels_data['lat'][:]
(lon, lat) = np.meshgrid(longitude, latitude)

# use PCA to reduce dimensions under the condition of reaching 80% of all the member infomation
exp_var = 0
n_pc = 0
while exp_var < 0.8:
    n_pc = n_pc + 1
    pca = PCA(n_components = n_pc)
    pca.fit(Geopotential_Isobaric_500_reshaped)
    exp_var = sum(pca.explained_variance_ratio_)

# get the transformed raw data in the dimension-reduced space    
pca_transformed_data = pca.transform(Geopotential_Isobaric_500_reshaped)

#get the time variable
times=Pressure_Levels_data.variables["time"]
#get the time number
arrDateEnd=nc.num2date(times[:],units=times.units)
#get the time in date format
dateEndDate = datetime.date(arrDateEnd[0]).strftime("%d %b %Y")
dateEndMin = datetime.date(arrDateEnd[0]).strftime("%H:%M")
dateStart=datetime.date(arrDateEnd[0])-timedelta(hours=120)
dateStartDate=dateStart.strftime("%d %b %Y")
dateStartMin=dateStart.strftime("%H:%M")

In [5]:
def Get_Subgrid(longitude,latitude,bottomLeft,topRight):
    lonStart=(np.abs(longitude - bottomLeft[0])).argmin()
    lonEnd=(np.abs(longitude - topRight[0])).argmin()
    latStart=(np.abs(latitude - bottomLeft[1])).argmin()
    latEnd=(np.abs(latitude - topRight[1])).argmin()
    
    return lonStart,lonEnd,latStart,latEnd

def Get_Meshgrid(lonStart,lonEnd,latStart,latEnd):
    long=longitude[lonStart:lonEnd+1]
    lati=latitude[latEnd:latStart+1]
    (lon,lat)=np.meshgrid(long,lati)
    
    return lon,lat

def Get_Area_Data(data,lonStart,lonEnd,latStart,latEnd):
    areaData=data[:,latEnd:latStart+1,lonStart:lonEnd+1]
    
    return areaData

def Reshape_New_Data(areaData):
    dim1,dim2,dim3=areaData.shape
    reshapedData=np.reshape(areaData,(dim1,dim2*dim3))
    return reshapedData

def PCA_Run(data):
    exp_var = 0
    n_pc = 0
    while exp_var < 0.8:
        n_pc = n_pc + 1
        pca = PCA(n_components = n_pc)
        pca.fit(data)
        exp_var = sum(pca.explained_variance_ratio_)
        
    return pca

def decompose_region(data,lon,lat,point1,point2):
    lonStart,lonEnd,latStart,latEnd = Get_Subgrid(lon,lat,point1,point2)
    secData = Get_Area_Data(data,lonStart,lonEnd,latStart,latEnd)
    reshapeData=Reshape_New_Data(secData)
    PCA_=PCA_Run(reshapeData)
    transformedData=PCA_.transform(reshapeData)
    
    return transformedData

In [11]:
from sklearn import preprocessing
#get the file path for loading, data file is under the same dir with the notebook
filename="20121017_12_ecmwf_forecast.PRESSURE_LEVELS.EUR_LL015.036.pl.nc"
foldername="ECWMF Datasets"
filepath=os.path.join(os.path.dirname(os.getcwd()),foldername,filename)
syn = nc.Dataset(filepath,"r")
#syn.variables
longitude1 = syn['lon'][:]
latitude1 = syn['lat'][:]
(lon1, lat1) = np.meshgrid(longitude1, latitude1)
syndata =syn.variables['Geopotential_isobaric'][:]/9.8
syndataiso1 = syndata[0,:,0,:,:]
syndataiso2 = syndata[0,:,1,:,:]
syndataiso3 = syndata[0,:,2,:,:]

synsmall=decompose_region(syndataiso1,longitude1,latitude1,[-10,35],[20,65])

realsmall=decompose_region(Geopotential_Isobaric_500,longitude,latitude,[-60,45],[-40,60])
realmid=decompose_region(Geopotential_Isobaric_500,longitude,latitude,[-60,30],[10,60])

In [13]:
def bandwidth_selection(data,sigThresh,outlierThresh):
    #constant
    step = 1
    n_clus = 0

    D = distance_matrix(X,X)
    minD = int(np.round(np.min(D[np.nonzero(D)])))
    if minD<0:
        minD=2
    maxD = int(np.round(np.max(D[np.nonzero(D)])))

    outlierVec=[]
    hCandidate=[]

    for i_h in range(int(np.round(minD/2)),maxD+1,step):
        meanShift=MeanShift(bandwidth=i_h).fit(X)
        labels=meanShift.labels_

        cl_sig=[k for k, v in Counter(labels).items() if v>=sigThresh]
        
        n_sig_new = len(cl_sig)
        numOutlierModes = len(set(labels))-len(cl_sig)
        
        if n_sig_new>n_clus:
            n_clus=n_sig_new
            hCandidate.append(i_h)
            outlierVec.append(numOutlierModes)
        elif n_sig_new==n_clus:
            hCandidate.append(i_h)
            outlierVec.append(numOutlierModes)
    
    h=hCandidate[-1]
    for i in range(len(hCandidate)):
        if outlierVec[i]<=outlierThresh:
            h=hCandidate[i]
            break
    return h

In [14]:
def knn_search(data,modes):
    modClosestMemID=np.zeros(len(modes))
    dist=np.zeros(len(modes))
    for idx,mode in enumerate(modes):
        if len(mode.shape)==1:
            mode=mode[np.newaxis,:]
        modeKNNid=np.argmin(euclidean_distances(data,mode))
        minDist=np.min(euclidean_distances(data,mode))
        modClosestMemID[idx]=modeKNNid
        dist[idx]=minDist
        
    return modClosestMemID,dist

def pairConn(X1,X2,lv,data,h):
    IDX,D=knn_search(X1,X2)
    #idx_2=np.where(D==D.min())
    #idx_1=IDX[idx_2]
    #minDist=D.min()
    meanDist=np.mean(D)
    
    isConn=1
    for one in range(len(X1)):
        for two in range(len(X2)):
            q1=X1[one,:]
            q2=X2[two,:]
            
            n_seq=20
            nl=np.zeros((n_seq,data.shape[1]))
            for i in range(data.shape[1]):
                s=q1[i]
                e=q2[i]
                if s==e:
                    nl[:,i]=np.zeros((n_seq,1))+s
                else:
                    nl[:,i]=np.linspace(s,e,n_seq)
            
            kde=KernelDensity(kernel="gaussian",bandwidth=h).fit(data)
            f_nl=np.exp(kde.score_samples(nl))
            
            if (np.sum(f_nl[f_nl<lv])>0) or (meanDist>2*h):
                isConn=0
                break
    return isConn

def cluConn(data,f,labels,lv,h):
    n_clu=len(set(labels))
    connMat=np.zeros((n_clu,n_clu))
    
    for i in range(n_clu-1):
        for j in range(i+1,n_clu):
            id_i=np.where(labels==i)
            id_j=np.where(labels==j)
            
            connMat[i,j]=pairConn(data[id_i],data[id_j],lv,data,h)
            connMat[j,i]=connMat[i,j]
            
    return connMat 

def high_density_clustering(data,h,sizeThresh):
    meanShift=MeanShift(bandwidth=h).fit(data)
    labels=meanShift.labels_
    modes=meanShift.cluster_centers_
    #modClosestMemID=knn_search(data,modes)
    n_clu=len(set(labels))
    
    kde=KernelDensity(kernel="gaussian",bandwidth=h).fit(data)
    f=np.exp(kde.score_samples(data))
    
    n_lv=20
    lv_seq=[i/n_lv*f.max() for i in range(1,n_lv+1)]
    
    #check the cluster info at each level: member, size and connectivity
    conn=np.empty((1,n_lv),dtype=object)
    l_conn=np.empty((1,n_lv),dtype=object)
    for i in range(n_lv):
        conn[0,i]=cluConn(data,f,labels,lv_seq[i],h)
    
    #check which clusters are connected at each level
    conn_mask=np.zeros(conn[0,0].shape)
    for i in reversed(range(n_lv)):
        lvl_conn=np.zeros(conn[0,i].shape)
        lvl_tmp=lvl_conn
        tmp=lvl_tmp
        for j in range(i,n_lv):
            tmp=tmp+conn[0,j]
        lvl_tmp=np.where(tmp!=1,lvl_tmp,1)
        
        #fing indices of newly connected clusters
        r_c,c_c=np.where(lvl_tmp==1)
        
        #for any pair of connected clusters, check if they are already connected
        for k in range(len(r_c)):
            if conn_mask[r_c[k],c_c[k]]==0:
                neighors1=np.where(conn_mask[r_c[k],:]==1)
                neighors2=np.where(conn_mask[c_c[k],:]==1)
                
                conn_mask[r_c[k],c_c[k]]=1
                conn_mask[c_c[k],r_c[k]]=1
                
                conn_mask[neighors1,c_c[k]]=1
                conn_mask[c_c[k],neighors1]=1
                conn_mask[neighors2,r_c[k]]=1
                conn_mask[r_c[k],neighors2]=1
                
                lvl_conn[r_c[k],c_c[k]]=1
                lvl_conn[c_c[k],r_c[k]]=1
        
        l_conn[0,i]=lvl_conn
        
    #cluster members by density
    upperLvlMembers=np.empty((n_lv,n_clu),dtype=object)
    lvlMembers=np.empty((n_lv,n_clu),dtype=object)
    upper_size=np.zeros((n_lv,n_clu))
      
    for i in range(n_lv-1):
        upper_idx=np.where(f>lv_seq[i])
        lvl_idx=np.where((f>lv_seq[i]) & (f<=lv_seq[i+1]))
        for j in range(n_clu):
            upperLvlMembers[i,j]=np.intersect1d(upper_idx,np.where(labels==j))
            lvlMembers[i,j]=np.intersect1d(lvl_idx,np.where(labels==j))
            upper_size[i,j]=len(upperLvlMembers[i,j])
            
    #select significant clusters just in case
    sig_clu=[k for k, v in Counter(labels).items() if v>=sizeThresh]
            
    return upperLvlMembers,lvlMembers,upper_size,sig_clu,f,conn

In [15]:
bandwidth_selection(synsmall,15,2)

6

In [16]:
bandwidth_selection(realsmall,15,2)

6

In [17]:
bandwidth_selection(realmid,15,2)

6